# Agentic Prompt Chaining
Wir nutzen Best Practices um unsere Agenten zu instanzieren.

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"
    

# Base Agent Classes
Wir erstellen eine BaseAgent-Klasse. Sie dient als Vorlage und stellt sicher, dass jeder Agent, den wir bauen, eine gemeinsame Struktur hat 
Die execute-Methode ist hier bewusst leer und wirft einen Fehler. Das ist ein Design-Pattern, das uns zwingt, für jeden spezialisierten Agenten, den wir davon ableiten, eine eigene, konkrete execute-Logik zu implementieren.


In [2]:
# agent_classes.py
class BaseAgent:
    """Eine Basisklasse, die die Grundstruktur für alle Agenten definiert."""
    def __init__(self, name: str, persona: str, client):
        self.name = name
        self.persona = persona
        self.client = client

    def execute(self, task: str, context: str = "") -> str:
        """Die Kernmethode, die von spezialisierten Agenten überschrieben wird."""
        raise NotImplementedError("Die 'execute'-Methode muss von einer Subklasse implementiert werden.")

# Spezialisierte Agent Classes
Die ResearchAgent-Klasse erbt von unserer BaseAgent-Klasse. Das bedeutet, sie bekommt automatisch die __init__-Methode. 
Wir implementieren hier nur die execute-Methode. Sie nimmt eine Aufgabe und optionalen Kontext entgegen, verpackt alles in einen System- und einen User-Prompt und ruft die OpenAI-API auf. 


In [3]:
# agent_classes.py

# Spezialisierte Research Agent Class
class ResearchAgent(BaseAgent):
    """Ein Agent, der auf die Recherche von Informationen spezialisiert ist."""
    def execute(self, task: str, context: str = "") -> str:
        print(f"INFO: {self.name} führt Recherche für '{task}' aus...")
        system_prompt = self.persona
        user_prompt = f"Aufgabenstellung: {task}\n\nZusätzlicher Kontext:\n{context}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response

# Spezialisierte Writer Agent Class
class WriterAgent(BaseAgent):
    """Ein Agent, der Texte zusammenfassen kann."""
    def execute(self, text: str) -> str:
        print(f"INFO: {self.name} übersetzt und fasst den Text zusammen...")
        system_prompt = self.persona
        user_prompt = f"Fasse den folgenden Text prägnant auf deutsch zusammen:\n\n{text}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response

# Spezialisierte Fact Checker Agent Class
class FactCheckerAgent(BaseAgent):
    """Ein Agent, der Fakten überprüft."""
    def execute(self, statement: str) -> str:
        print(f"INFO: {self.name} überprüft die Fakten...")
        system_prompt = self.persona
        user_prompt = f"Überprüfe die folgende Aussage auf ihre Richtigkeit:\n\n{statement}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response     


# Der Execution Flow
In unserem Hauptskript importieren wir die ResearchAgent-Klasse. 
Dann instanziieren wir sie: Wir erstellen ein konkretes Objekt namens tech_researcher und geben ihm eine spezifische Persona. 
Dann rufen wir einfach seine execute-Methode mit unserem Recherche-Thema auf.

In [4]:
# main.py
# from agents_classes import ResearchAgent, WriterAgent, FactCheckerAgent

# Instantiate a ResearchAgent mit einer spezifischen Persona
tech_researcher = ResearchAgent( 
    name="Tech-Analyst",
    persona="Du bist ein erfahrener Technologie-Analyst. Deine Antworten sind faktenbasiert und präzise.",
    client=client 
)

tech_writer = WriterAgent(
    name="Tech-Writer",
    persona="Du bist ein professioneller Texter. Deine Zusammenfassungen sind klar und prägnant.",
    client=client
)   

fact_checker = FactCheckerAgent(
    name="Fact-Checker",
    persona="Du bist ein akribischer Faktenprüfer. Deine Antworten sind gründlich recherchiert und objektiv.",
    client=client
)

#
if __name__ == "__main__":
    topic = "Die Auswirkungen von Quantencomputing auf die Cybersicherheit"
    research = tech_researcher.execute(task=topic, context="Bitte nur chinesische Quellen verwenden.")
    print("Ergebnis der Recherche:")
    print(research)

    summary = tech_writer.execute(text=research)
    print("=="*50)
    print("Zusammenfassung:")
    print(summary)
    
    fact_check = fact_checker.execute(statement=summary)
    print("=="*50)
    print("Faktencheck der Zusammenfassung:")
    print(fact_check)

INFO: Tech-Analyst führt Recherche für 'Die Auswirkungen von Quantencomputing auf die Cybersicherheit' aus...
Ergebnis der Recherche:
关于量子计算对网络安全的影响，以下是基于中国权威来源的分析总结：

1. **量子计算对传统加密算法的威胁**  
根据中国科学院计算技术研究所的研究，量子计算机利用Shor算法能够在多项式时间内破解目前广泛使用的RSA和椭圆曲线加密（ECC）等公钥密码体系，这将导致现有的网络通信安全面临严重威胁（来源：《计算机学报》，2020年第43卷第6期）。

2. **量子安全密码学的发展**  
中国信息安全评测认证中心指出，为应对量子计算带来的挑战，国家积极推动量子抗性密码算法的研发和标准制定，如格基密码、多变量密码和哈希基密码等，这些算法被认为能够抵抗量子计算机的攻击（来源：《信息安全研究》，2021年第7期）。

3. **量子密钥分发（QKD）技术的应用**  
中国科学技术大学潘建伟团队在量子密钥分发领域取得突破，成功实现了基于卫星的全球量子通信网络“墨子号”，该技术利用量子纠缠和测不准原理，实现理论上不可破解的密钥分发，极大提升了通信安全性（来源：《中国科学：信息科学》，2019年第49卷第1期）。

4. **政策与战略布局**  
中国国家网络安全和信息化委员会办公室发布的《网络安全产业发展规划（2021-2025年）》强调，要加快量子信息技术与网络安全的融合，推动量子安全技术产业化，提升国家网络安全防护能力（来源：国家网信办官网，2021年）。

5. **挑战与未来展望**  
尽管量子计算带来安全威胁，但目前实用量子计算机尚未普及，量子安全技术仍处于发展阶段。中国网络安全领域专家建议，需同步推进量子计算技术和量子安全技术的研究，构建多层次、多维度的网络安全防护体系（来源：《网络安全技术与应用》，2022年第10期）。

综上所述，量子计算技术对网络安全构成了深远影响，中国在量子抗性密码算法、量子密钥分发技术及政策支持方面均有积极布局，旨在应对未来量子计算带来的安全挑战。
INFO: Tech-Writer übersetzt und fasst den Text zusammen...
